In [1]:
import plotly.graph_objs as go
import plotly.express as px
from plotly import tools, subplots
import numpy as np
import pandas as pd
import ipywidgets as widgets
import math
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from ipywidgets import interact, interactive, fixed, interact_manual
init_notebook_mode(connected=True)
import scipy.io as sio
import pickle

## T1 values Before B1 correction

In [20]:
derivativesDir = '/Users/agah/Desktop/KuzuData/PHD_DATA/ds-venus/derivatives/qMRLab/sub-phantom/'

spheres = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
refMean = [1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]


sessions = ['rth750retest','rth750test','rthPRIretest','rthPRItest','rthSKYretest','rthSKYtest',
        'vendor750retest','vendor750test','vendorPRIretest','vendorPRItest','vendorSKYretest','vendorSKYtest']

convert_tag = {'rth750':'G1<sub>neutral</sub>','rthPRI':'S1<sub>neutral</sub>','rthSKY':'S2<sub>neutral</sub>',
               'vendor750':'G1<sub>native</sub>','vendorPRI':'S1<sub>native</sub>','vendorSKY':'S2<sub>native</sub>'
              }
# 'rth750':
colors = {'rth750':'rgba(255,8,30,1)','rthPRI':'rgba(255,117,0,1)','rthSKY':'rgba(255,220,0,1)',
         'vendor750':'rgba(13,186,233,1)','vendorPRI':'rgba(2.5,187,180,1)','vendorSKY':'rgba(0,248,205,1)'}

colors_line = {'rth750':'rgba(255,8,30,0.2)','rthPRI':'rgba(255,117,0,0.2)','rthSKY':'rgba(255,220,0,0.2)',
         'vendor750':'rgba(13,186,233,0.2)','vendorPRI':'rgba(2.5,187,180,0.2)','vendorSKY':'rgba(0,248,205,0.2)'}


line_dash = {'rth750':'dash','rthPRI':'dash','rthSKY':'dash',
         'vendor750':'solid','vendorPRI':'solid','vendorSKY':'solid'}

line_sym = {'rth750':'circle','rthPRI':'circle','rthSKY':'circle',
         'vendor750':'square','vendorPRI':'square','vendorSKY':'square'}

df = pd.DataFrame()

fig = go.Figure()


for jj in range(0,len(sessions),2):
    avg_array  = []
    std_array  =  []
    for ii in range(len(spheres)):
        # Each one of these contains B1, MTsat, T1 and T1cor
        cur_sphere_test = derivativesDir + 'ses-' + sessions[jj] + '/stat/sub-phantom_ses-' + sessions[jj] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_sphere_retest = derivativesDir + 'ses-' + sessions[jj+1] + '/stat/sub-phantom_ses-' + sessions[jj+1] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_contents_test = sio.loadmat(cur_sphere_test)
        cur_contents_retest = sio.loadmat(cur_sphere_retest)
        cur_tag = sessions[jj][0:-6]
        avg_array.append(np.mean([cur_contents_test['T1'].flatten().mean(),cur_contents_retest['T1'].flatten().mean()]))
        std_array.append(np.mean([cur_contents_test['T1'].flatten().std(),cur_contents_retest['T1'].flatten().std()]))
    df[cur_tag + '_mean'] = avg_array
    df[cur_tag + '_std'] = std_array
    upper = list(np.squeeze(np.array(avg_array)) + np.squeeze(np.array(std_array)))
    lower = list(np.squeeze(np.array(avg_array)) - np.squeeze(np.array(std_array)))
    # Add std lines
    fig.add_trace(go.Scatter(
    x= spheres + spheres[::-1],
    y=upper+lower[::-1],
    fill='toself',
    fillcolor= colors_line[cur_tag],
    line_color='rgba(255,0,0,0)',
    name= convert_tag[cur_tag],
    ))
    # Add markers
    fig.add_trace(go.Scatter(
    x=spheres, y=avg_array,
    line_color=colors[cur_tag],
    name= convert_tag[cur_tag],
    marker = dict(size=20,symbol=line_sym[cur_tag],line=dict(color='black',width=1.5),opacity=0.85),
    line=dict(width=2, dash=line_dash[cur_tag]),
    line_shape='spline'
    ))



# Theme change
fig.update_layout(height=800, width=1300, paper_bgcolor='#000000',plot_bgcolor='#000000')

#fig.update_layout(height=800, width=1000)
fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")))
axis_template = dict(linecolor = 'black', showticklabels = True,
             tickfont=dict(color="white"), gridcolor = 'black',zerolinecolor = 'rgb(40,40,40)' )
fig.update_xaxes(axis_template)

# Theme change
fig.update_yaxes(axis_template,gridcolor = 'rgb(100,100,100)')

plot_y = [2.5,1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]

fig.update_yaxes(tickvals=np.round(plot_y,2),range=[0,2.38])
fig.update_xaxes(tickvals=spheres)
fig.update_layout(title='',title_font_color="white",margin=dict(l=0,r=0,t=0,b=0))


avg_array = [2.03,1.39,0.93,0.63, 0.42, 0.29, 0.22, 0.18, 0.14, 0.11 ]
std_array = [0.09, 0.04, 0.02, 0.01, 0.02,0.01, 0.01, 0.01, 0.01, 0.01 ]
upper = list(np.squeeze(np.array(avg_array)) + np.squeeze(np.array(std_array)))
lower = list(np.squeeze(np.array(avg_array)) - np.squeeze(np.array(std_array)))
# Add std lines
fig.add_trace(go.Scatter(
x= spheres + spheres[::-1],
y=upper+lower[::-1],
fill='toself',
fillcolor= colors_line['rth750'],
line_color='rgba(255,0,0,0)',
name= convert_tag['rth750'],
))
# Add markers
fig.add_trace(go.Scatter(
x=spheres, y=avg_array,
line_color=colors['rth750'],
name= 'rth750new',
marker = dict(size=20,symbol=line_sym['rth750'],line=dict(color='black',width=1.5),opacity=0.85),
line=dict(width=2, dash=line_dash['rth750']),
line_shape='spline'
))


fig.add_trace(go.Scatter(
    x=spheres, y=refMean,
    line_color='white',
    mode = 'markers',
    name = 'Reference',
    marker = dict(size=25,symbol='cross-open',line = dict(width=2))

))



plot(fig)

'temp-plot.html'

In [ ]:
## AVG

In [2]:
derivativesDir = '/Users/agah/Desktop/KuzuData/PHD_DATA/ds-venus/derivatives/qMRLab/sub-phantom/'

spheres = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
refMean = [1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]


sessions = ['rth750retest','rth750test','rthPRIretest','rthPRItest','rthSKYretest','rthSKYtest',
        'vendor750retest','vendor750test','vendorPRIretest','vendorPRItest','vendorSKYretest','vendorSKYtest']
df = pd.DataFrame()

fig = go.Figure()

avg_array  = []
std_array  =  []
for ii in range(len(spheres)):
    # Each one of these contains B1, MTsat, T1 and T1cor
    cur_region_rth750 = derivativesDir + 'ses-' + sessions[0] + '/stat/sub-phantom_ses-' + sessions[0] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
    cur_region_rthPRI = derivativesDir + 'ses-' + sessions[2] + '/stat/sub-phantom_ses-' + sessions[2] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
    cur_region_rthSKY = derivativesDir + 'ses-' + sessions[4] + '/stat/sub-phantom_ses-' + sessions[4] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'



    cur_contents_rth750 = sio.loadmat(cur_region_rth750)
    cur_contents_rthPRI = sio.loadmat(cur_region_rthPRI)
    cur_contents_rthSKY = sio.loadmat(cur_region_rthSKY)
    
    avg_array.append(np.mean(np.array([cur_contents_rth750['T1'].flatten().mean(),cur_contents_rthPRI['T1'].flatten().mean(),cur_contents_rthSKY['T1'].flatten().mean(),]),axis=0))
    std_array.append(np.std(np.array([cur_contents_rth750['T1'].flatten().std(),cur_contents_rthPRI['T1'].flatten().std(),cur_contents_rthSKY['T1'].flatten().std(),]),axis=0))
    #cur_contents_avg = np.delete(cur_contents_avg, np.where(cur_contents_avg > 5))
    cur_tag = 'neutral'
df[cur_tag + '_mean'] = avg_array
df[cur_tag + '_std'] = std_array
upper = list(np.squeeze(np.array(avg_array)) + np.squeeze(np.array(std_array)))
lower = list(np.squeeze(np.array(avg_array)) - np.squeeze(np.array(std_array)))
# Add std lines
fig.add_trace(go.Scatter(
x= spheres + spheres[::-1],
y=upper+lower[::-1],
fill='toself',
fillcolor= 'rgba(247,202,171,0.3)',
line_color='rgba(255,0,0,0)',
name= 'neutral'
))
# Add markers
fig.add_trace(go.Scatter(
    x=spheres, y=refMean,
    line_color='black',
    mode = 'markers',
    name = 'Reference',
    marker = dict(size=25,symbol='cross-open',line = dict(width=2))

))
fig.add_trace(go.Scatter(
x=spheres, y=avg_array,
line_color='rgba(247,0,0,1)',
name= 'neutral',
marker = dict(size=20,symbol='circle',line=dict(color='black',width=1.5),opacity=0.9),
line=dict(width=2, dash='dash'),
line_shape='spline'
))




In [3]:
derivativesDir = '/Users/agah/Desktop/KuzuData/PHD_DATA/ds-venus/derivatives/qMRLab/sub-phantom/'

spheres = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
refMean = [1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]


sessions = ['rth750retest','rth750test','rthPRIretest','rthPRItest','rthSKYretest','rthSKYtest',
        'vendor750retest','vendor750test','vendorPRIretest','vendorPRItest','vendorSKYretest','vendorSKYtest']
df = pd.DataFrame()

#fig = go.Figure() PREV CELL

avg_array  = []
std_array  =  []
for ii in range(len(spheres)):
    # Each one of these contains B1, MTsat, T1 and T1cor
    cur_region_rth750 = derivativesDir + 'ses-' + sessions[6] + '/stat/sub-phantom_ses-' + sessions[6] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
    cur_region_rthPRI = derivativesDir + 'ses-' + sessions[8] + '/stat/sub-phantom_ses-' + sessions[8] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
    cur_region_rthSKY = derivativesDir + 'ses-' + sessions[10] + '/stat/sub-phantom_ses-' + sessions[10] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'



    cur_contents_rth750 = sio.loadmat(cur_region_rth750)
    cur_contents_rthPRI = sio.loadmat(cur_region_rthPRI)
    cur_contents_rthSKY = sio.loadmat(cur_region_rthSKY)
    
    avg_array.append(np.mean(np.array([cur_contents_rth750['T1'].flatten().mean(),cur_contents_rthPRI['T1'].flatten().mean(),cur_contents_rthSKY['T1'].flatten().mean(),]),axis=0))
    std_array.append(np.std(np.array([cur_contents_rth750['T1'].flatten().std(),cur_contents_rthPRI['T1'].flatten().std(),cur_contents_rthSKY['T1'].flatten().std(),]),axis=0))
    #cur_contents_avg = np.delete(cur_contents_avg, np.where(cur_contents_avg > 5))
    cur_tag = 'native'
df[cur_tag + '_mean'] = avg_array
df[cur_tag + '_std'] = std_array
upper = list(np.squeeze(np.array(avg_array)) + np.squeeze(np.array(std_array)))
lower = list(np.squeeze(np.array(avg_array)) - np.squeeze(np.array(std_array)))
# Add std lines
fig.add_trace(go.Scatter(
x= spheres + spheres[::-1],
y=upper+lower[::-1],
fill='toself',
fillcolor= 'rgba(0,0,255,0.8)',
line_color='rgba(255,0,0,0)',
name= 'native'
))
# Add markers
fig.add_trace(go.Scatter(
x=spheres, y=avg_array,
line_color='rgba(0,0,255,0.8)',
name= 'native',
marker = dict(size=20,symbol='square',line=dict(color='black',width=1.5),opacity=0.9),
line=dict(width=2, dash='solid'),
line_shape='spline'
))
fig.update_layout(height=700, width=800)


fig

In [4]:
fig.update_layout(height=700, width=800, paper_bgcolor='#000000',plot_bgcolor='#000000')
#fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")))
axis_template = dict(linecolor = 'black', showticklabels = True,
             tickfont=dict(color="white"), gridcolor = 'rgb(20,20,20)',zerolinecolor = 'rgb(40,40,40)' )
fig.update_xaxes(axis_template)
fig.update_yaxes(axis_template,gridcolor = 'rgb(100,100,100)')

plot_y = [2.5,1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]

fig.update_yaxes(tickvals=np.round(plot_y,2),range=[0,2.38])
fig.update_xaxes(tickvals=spheres)
fig.update_layout(title='',title_font_color="white",margin=dict(l=0,r=0,t=0,b=0))

plot(fig)

'temp-plot.html'

## T1 values After B1 correction

In [ ]:
derivativesDir = '/Users/agah/Desktop/KuzuData/PHD_DATA/ds-venus/derivatives/qMRLab/sub-phantom/'

spheres = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
refMean = [1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]


sessions = ['rth750retest','rth750test','rthPRIretest','rthPRItest','rthSKYretest','rthSKYtest',
            'vendorPRIretest','vendorPRItest','vendorSKYretest','vendorSKYtest']

convert_tag = {'rth750':'G1<sub>neutral</sub>','rthPRI':'S1<sub>neutral</sub>','rthSKY':'S2<sub>neutral</sub>',
               'vendor750':'G1<sub>native</sub>','vendorPRI':'S1<sub>native</sub>','vendorSKY':'S2<sub>native</sub>'
              }
# 'rth750':
colors = {'rth750':'rgba(255,8,30,1)','rthPRI':'rgba(255,117,0,1)','rthSKY':'rgba(255,220,0,1)',
         'vendor750':'rgba(13,186,233,1)','vendorPRI':'rgba(2.5,187,180,1)','vendorSKY':'rgba(0,248,205,1)'}

colors_line = {'rth750':'rgba(255,8,30,0.2)','rthPRI':'rgba(255,117,0,0.1)','rthSKY':'rgba(255,220,0,0.1)',
         'vendor750':'rgba(13,186,233,0.1)','vendorPRI':'rgba(2.5,187,180,0.1)','vendorSKY':'rgba(0,248,205,0.11)'}


line_dash = {'rth750':'dash','rthPRI':'dash','rthSKY':'dash',
         'vendor750':'solid','vendorPRI':'solid','vendorSKY':'solid'}

line_sym = {'rth750':'circle','rthPRI':'circle','rthSKY':'circle',
         'vendor750':'square','vendorPRI':'square','vendorSKY':'square'}

df = pd.DataFrame()

fig = go.Figure()


for jj in range(0,len(sessions),2):
    avg_array  = []
    std_array  =  []
    for ii in range(len(spheres)):
        # Each one of these contains B1, MTsat, T1 and T1cor
        cur_sphere_test = derivativesDir + 'ses-' + sessions[jj] + '/stat/sub-phantom_ses-' + sessions[jj] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_sphere_retest = derivativesDir + 'ses-' + sessions[jj+1] + '/stat/sub-phantom_ses-' + sessions[jj+1] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_contents_test = sio.loadmat(cur_sphere_test)
        cur_contents_retest = sio.loadmat(cur_sphere_retest)
        cur_tag = sessions[jj][0:-6]
        avg_array.append(np.mean([cur_contents_test['T1cor'].flatten().mean(),cur_contents_retest['T1cor'].flatten().mean()]))
        std_array.append(np.mean([cur_contents_test['T1cor'].flatten().std(),cur_contents_retest['T1cor'].flatten().std()]))
    df[cur_tag + '_mean'] = avg_array
    df[cur_tag + '_std'] = std_array
    upper = list(np.squeeze(np.array(avg_array)) + np.squeeze(np.array(std_array)))
    lower = list(np.squeeze(np.array(avg_array)) - np.squeeze(np.array(std_array)))
    # Add std lines
    fig.add_trace(go.Scatter(
    x= spheres + spheres[::-1],
    y=upper+lower[::-1],
    fill='toself',
    fillcolor= colors_line[cur_tag],
    line_color='rgba(255,0,0,0)',
    name= convert_tag[cur_tag],
    ))
    # Add markers
    fig.add_trace(go.Scatter(
    x=spheres, y=avg_array,
    line_color=colors[cur_tag],
    name= convert_tag[cur_tag],
    marker = dict(size=20,symbol=line_sym[cur_tag],line=dict(color='black',width=1.5),opacity=0.75),
    line=dict(width=2, dash=line_dash[cur_tag]),
    line_shape='spline'
    ))

fig.add_trace(go.Scatter(
    x=spheres, y=refMean,
    line_color='white',
    mode = 'markers',
    name = 'Reference',
    marker = dict(size=25,symbol='cross-open',line = dict(width=2))

))

fig.update_layout(height=800, width=1000, paper_bgcolor='#000000',plot_bgcolor='#000000')
fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")))
axis_template = dict(linecolor = 'black', showticklabels = True,
             tickfont=dict(color="white"), gridcolor = 'rgb(20,20,20)',zerolinecolor = 'rgb(40,40,40)' )
fig.update_xaxes(axis_template)
fig.update_yaxes(axis_template,gridcolor = 'rgb(100,100,100)')

plot_y = [2.5,1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]

fig.update_yaxes(tickvals=np.round(plot_y,2),range=[0,2.38])
fig.update_xaxes(tickvals=spheres)
fig.update_layout(title='',title_font_color="white",margin=dict(l=0,r=0,t=0,b=0))

plot(fig)

## Percent dif Before B1 correction

In [ ]:
derivativesDir = '/Users/agah/Desktop/KuzuData/PHD_DATA/ds-venus/derivatives/qMRLab/sub-phantom/'

spheres = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
refMean = [1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]


sessions = ['rth750retest','rth750test','rthPRIretest','rthPRItest','rthSKYretest','rthSKYtest',
            'vendor750retest', 'vendor750test','vendorPRIretest','vendorPRItest','vendorSKYretest','vendorSKYtest']

convert_tag = {'rth750':'G1<sub>neutral</sub>','rthPRI':'S1<sub>neutral</sub>','rthSKY':'S2<sub>neutral</sub>',
               'vendor750':'G1<sub>native</sub>','vendorPRI':'S1<sub>native</sub>','vendorSKY':'S2<sub>native</sub>'
              }
# 'rth750':
colors = {'rth750':'rgba(255,8,30,1)','rthPRI':'rgba(255,117,0,1)','rthSKY':'rgba(255,220,0,1)',
         'vendor750':'rgba(13,186,233,1)','vendorPRI':'rgba(2.5,187,180,1)','vendorSKY':'rgba(0,248,205,1)'}

colors_line = {'rth750':'rgba(255,8,30,0.2)','rthPRI':'rgba(255,117,0,0.1)','rthSKY':'rgba(255,220,0,0.1)',
         'vendor750':'rgba(13,186,233,0.1)','vendorPRI':'rgba(2.5,187,180,0.1)','vendorSKY':'rgba(0,248,205,0.11)'}


line_dash = {'rth750':'dash','rthPRI':'dash','rthSKY':'dash',
         'vendor750':'solid','vendorPRI':'solid','vendorSKY':'solid'}

line_sym = {'rth750':'circle','rthPRI':'circle','rthSKY':'circle',
         'vendor750':'square','vendorPRI':'square','vendorSKY':'square'}


df = pd.DataFrame()

fig = go.Figure()


for jj in range(0,len(sessions),2):
    avg_array  = []
    std_array  =  []
    pctdif = []
    for ii in range(len(spheres)):
        # Each one of these contains B1, MTsat, T1 and T1cor
        cur_sphere_test = derivativesDir + 'ses-' + sessions[jj] + '/stat/sub-phantom_ses-' + sessions[jj] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_sphere_retest = derivativesDir + 'ses-' + sessions[jj+1] + '/stat/sub-phantom_ses-' + sessions[jj+1] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_contents_test = sio.loadmat(cur_sphere_test)
        cur_contents_retest = sio.loadmat(cur_sphere_retest)
        cur_tag = sessions[jj][0:-6]
        cur_val = np.mean([cur_contents_test['T1'].flatten().mean(),cur_contents_retest['T1'].flatten().mean()])
        pctdif.append((abs(cur_val-refMean[ii])/(np.mean([cur_val,refMean[ii]])))*100) 
        std_array.append(np.mean([cur_contents_test['T1'].flatten().std(),cur_contents_retest['T1'].flatten().std()]))
    #df[cur_tag + '_mean'] = avg_array
    #df[cur_tag + '_std'] = std_array
    #upper = list(np.squeeze(np.array(avg_array)) + np.squeeze(np.array(std_array)))
    #lower = list(np.squeeze(np.array(avg_array)) - np.squeeze(np.array(std_array)))
    # Add markers
    fig.add_trace(go.Scatter(
    x=spheres, y=pctdif,
    line_color=colors[cur_tag],
    name= convert_tag[cur_tag],
    marker = dict(size=20,symbol=line_sym[cur_tag],line=dict(color='black',width=1.5),opacity=0.75),
    line=dict(width=2, dash=line_dash[cur_tag]),
    line_shape='spline'
    ))


fig.update_layout(height=500, width=1000, paper_bgcolor='#000000',plot_bgcolor='#000000')
fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")),margin=dict(l=0,r=0,t=0,b=0))
axis_template = dict(linecolor = 'black', showticklabels = True,
             tickfont=dict(color="white"), gridcolor = 'rgb(20,20,20)',zerolinecolor = 'rgb(40,40,40)' )
fig.update_xaxes(axis_template)
fig.update_yaxes(axis_template,gridcolor = 'rgb(60,60,60)')

#plot_y = [2.5,1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]

fig.update_yaxes(range=[-1,55])
fig.update_xaxes(tickvals=spheres)

fig.update_layout(title_font_color="white",)

plot(fig)

## Percent error after B1

In [ ]:
derivativesDir = '/Users/agah/Desktop/KuzuData/PHD_DATA/ds-venus/derivatives/qMRLab/sub-phantom/'

spheres = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
refMean = [1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]


sessions = ['rth750retest','rth750test','rthPRIretest','rthPRItest','rthSKYretest','rthSKYtest',
            'vendorPRIretest','vendorPRItest','vendorSKYretest','vendorSKYtest']

convert_tag = {'rth750':'G1<sub>neutral</sub>','rthPRI':'S1<sub>neutral</sub>','rthSKY':'S2<sub>neutral</sub>',
               'vendor750':'G1<sub>native</sub>','vendorPRI':'S1<sub>native</sub>','vendorSKY':'S2<sub>native</sub>'
              }
# 'rth750':
colors = {'rth750':'rgba(255,8,30,1)','rthPRI':'rgba(255,117,0,1)','rthSKY':'rgba(255,220,0,1)',
         'vendor750':'rgba(0,75,255,1)','vendorPRI':'rgba(24,231,234,1)','vendorSKY':'rgba(24,234,141,1)'}

colors_line = {'rth750':'rgba(255,8,30,0.35)','rthPRI':'rgba(255,117,0,0.35)','rthSKY':'rgba(255,220,0,0.35)',
         'vendor750':'rgba(0,75,255,0.35)','vendorPRI':'rgba(24,231,234,0.35)','vendorSKY':'rgba(24,234,141,0.35)'}

df = pd.DataFrame()

fig = go.Figure()


for jj in range(0,len(sessions),2):
    avg_array  = []
    std_array  =  []
    pctdif = []
    for ii in range(len(spheres)):
        # Each one of these contains B1, MTsat, T1 and T1cor
        cur_sphere_test = derivativesDir + 'ses-' + sessions[jj] + '/stat/sub-phantom_ses-' + sessions[jj] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_sphere_retest = derivativesDir + 'ses-' + sessions[jj+1] + '/stat/sub-phantom_ses-' + sessions[jj+1] + '_desc-sphere' + str(spheres[ii]) + '_metrics.mat'
        cur_contents_test = sio.loadmat(cur_sphere_test)
        cur_contents_retest = sio.loadmat(cur_sphere_retest)
        cur_tag = sessions[jj][0:-6]
        cur_val = np.mean([cur_contents_test['T1cor'].flatten().mean(),cur_contents_retest['T1cor'].flatten().mean()])
        pctdif.append((abs(cur_val-refMean[ii])/(np.mean([cur_val,refMean[ii]])))*100) 
        std_array.append(np.mean([cur_contents_test['T1cor'].flatten().std(),cur_contents_retest['T1cor'].flatten().std()]))
    #df[cur_tag + '_mean'] = avg_array
    #df[cur_tag + '_std'] = std_array
    #upper = list(np.squeeze(np.array(avg_array)) + np.squeeze(np.array(std_array)))
    #lower = list(np.squeeze(np.array(avg_array)) - np.squeeze(np.array(std_array)))
    # Add markers
    fig.add_trace(go.Scatter(
    x=spheres, y=pctdif,
    line_color=colors[cur_tag],
    name= convert_tag[cur_tag],
    marker = dict(size=10),
    ))


fig.update_layout(height=500, width=1000, paper_bgcolor='#000000',plot_bgcolor='#000000')
fig.update_layout(legend=dict(x=1, y=1,tracegroupgap=300,font=dict(color="white")))
axis_template = dict(linecolor = 'black', showticklabels = True,
             tickfont=dict(color="white"), gridcolor = 'rgb(20,20,20)',zerolinecolor = 'rgb(40,40,40)' )
fig.update_xaxes(axis_template)
fig.update_yaxes(axis_template,gridcolor = 'rgb(60,60,60)')

#plot_y = [2.5,1.989,1.454,0.9841,0.706,0.4967,0.3515,0.24713,0.1753,0.1259,0.089]

fig.update_yaxes(range=[-1,45])
fig.update_xaxes(tickvals=spheres)

fig.update_layout(title='Percent difference | B1 corrected',title_font_color="white",)


plot(fig)